# Classificador

Neste Workshop, vamos aprender como criar um classificador que diz quais músicas são da Rihanna e quais são da Beyoncé utilizando aprendizado de máquina, mais especificamente *Aprendizado Supervisionado*, uma das áreas de Machine Learning.

Então vamos lá!



## Importando os dados

O primeiro passo é importar seus dados, no caso nosso Dataframe (como chamamos a *tabela* que guarda as informações que usaremos). Fazemos isso com a biblioteca `pandas`.

Com a função `read_csv` lemos nosso arquivo e guardamos ele na váriavel `df`.



In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('beyonce_rihanna.csv', index_col=0)

# vamos explorar nosso dataframe olhando apenas as primeiras linhas com a função abaixo:
df.head()

,Nome da Música,link,album,letra,artista
0,'03 Bonnie & Clyde,/beyonce/03-bonnie-clyde.html,I Am... Yours: An Intimate Performance at Wynn...,Jay-z Uh-uh-uh You ready b? Let's go get 'em. ...,Beyoncé
1,***Flawless (Feat. Chimamanda Ngozi Adichie),/beyonce/flawless-feat-chimamanda-ngozi-adichi...,BEYONCÉ,Your challengers are a young group from Housto...,Beyoncé
2,***Flawless (Feat. Nicki Minaj),/beyonce/flawless-feat-nicki-minaj.html,BEYONCÉ [Platinum Edition],"Dum-da-de-da Do, do, do, do, do, do (Coming do...",Beyoncé
3,1+1,/beyonce/11.html,BEYONCÉ [Platinum Edition],If I ain't got nothing I got you If I ain't go...,Beyoncé
4,6 Inch (Feat. The Weeknd),/beyonce/6-inch-feat-the-weeknd.html,LEMONADE,Six inch heels She walked in the club like nob...,Beyoncé


## Pré-processamentos

Antes de partir para o aprendizado de máquina, precisamos preparar nosso texto. Fazemos isso porque, para a máquina, algumas palavras ou estruturas do nosso texto não importam e não fazem diferença. 
São muitos os métodos de pré-processamento, mas aqui vamos realizar apenas alguns: 
* Tokenização
* Remover stopwords
* Deixar todo o texto em minúsculo
* Selecionar apenas letras com REGEX
* Lemmatização

In [3]:
# Utilizando uma música como exemplo
exemplo = df['letra'][10]
exemplo

"Here I am Looking in the mirror An open face, the pain erased Now the sky is clearer I can see the sun Now that all is, all is said and done, oh  There you are Always strong when I need you You let me give And now I live, fearless and protected With the one I will love After all is, all is said and done  I once believed that hearts were made to bleed (Inside I once believed that hearts were made to bleed, oh baby) But now I'm not afraid to say I need you, I need you so stay with me  These precious (precious) hours (yeah) Greet each dawn in open arms And dream, into tomorrow  Where there's only love After all is, all is said and done  (Yeah baby) Oh baby (Inside I once believed, That hearts were meant to bleed)  (I'll never) I'll never be afraid to say I need you, I need you, so here  Here we are in the still of this moment Fear is gone, hope lives on  We found our happy ending For there's only love (only love) And this sweet, sweet love After all is, all is said and done  Yeah baby af

### Tokenização 

Uma parte importante no pré-processamento de um texto é a tokenização. Isto é, transformar elementos do seu texto em tokens, ou seja, strings dentro de uma lista  -  ou, se você não tiver conhecimento de python, transformar todas as palavras do texto em elementos individuais separados por aspas. 
Podemos tokenizar palavras com `word_tokenize`, essa função recebe o texto como argumento e retorna todas as palavras do texto em forma de tokens.



In [4]:
from nltk.tokenize import word_tokenize

In [5]:
#Tokenizando a primeira música
tokens = word_tokenize(exemplo)
tokens

['Here',
 'I',
 'am',
 'Looking',
 'in',
 'the',
 'mirror',
 'An',
 'open',
 'face',
 ',',
 'the',
 'pain',
 'erased',
 'Now',
 'the',
 'sky',
 'is',
 'clearer',
 'I',
 'can',
 'see',
 'the',
 'sun',
 'Now',
 'that',
 'all',
 'is',
 ',',
 'all',
 'is',
 'said',
 'and',
 'done',
 ',',
 'oh',
 'There',
 'you',
 'are',
 'Always',
 'strong',
 'when',
 'I',
 'need',
 'you',
 'You',
 'let',
 'me',
 'give',
 'And',
 'now',
 'I',
 'live',
 ',',
 'fearless',
 'and',
 'protected',
 'With',
 'the',
 'one',
 'I',
 'will',
 'love',
 'After',
 'all',
 'is',
 ',',
 'all',
 'is',
 'said',
 'and',
 'done',
 'I',
 'once',
 'believed',
 'that',
 'hearts',
 'were',
 'made',
 'to',
 'bleed',
 '(',
 'Inside',
 'I',
 'once',
 'believed',
 'that',
 'hearts',
 'were',
 'made',
 'to',
 'bleed',
 ',',
 'oh',
 'baby',
 ')',
 'But',
 'now',
 'I',
 "'m",
 'not',
 'afraid',
 'to',
 'say',
 'I',
 'need',
 'you',
 ',',
 'I',
 'need',
 'you',
 'so',
 'stay',
 'with',
 'me',
 'These',
 'precious',
 '(',
 'precious',
 ')

### Selecionando apenas as letras e deixando todas em minúsculas

Para a máquina, pontuações não são necessárias, por isso um pré-processamento necessário é selecionar apenas as letras de um texto. 

Porém, **antes disso** precisamos deixar todas as letras em minúsculo, não somente porque isso facilita a aplicação do REGEX, mas também porque a máquina tende a interpretar palavras com letras maiúsculas e minúsculas como sendo diferentes. Por exemplo, Beyoncé e beyoncé podem ser interpretadas como palavras distintas. Então vamos deixar as letras minúsculas com a função `.lower`.

Feito isso, podemos selecionar apenas as letras com REGEX, mais especificamente com a função `re.findall`, que, além de retornar apenas as letras, já tokeniza o texto para você! 

 

In [6]:
import re
letras = re.findall(r'\b[A-zÀ-úü]+\b', exemplo.lower())
letras

['here',
 'i',
 'am',
 'looking',
 'in',
 'the',
 'mirror',
 'an',
 'open',
 'face',
 'the',
 'pain',
 'erased',
 'now',
 'the',
 'sky',
 'is',
 'clearer',
 'i',
 'can',
 'see',
 'the',
 'sun',
 'now',
 'that',
 'all',
 'is',
 'all',
 'is',
 'said',
 'and',
 'done',
 'oh',
 'there',
 'you',
 'are',
 'always',
 'strong',
 'when',
 'i',
 'need',
 'you',
 'you',
 'let',
 'me',
 'give',
 'and',
 'now',
 'i',
 'live',
 'fearless',
 'and',
 'protected',
 'with',
 'the',
 'one',
 'i',
 'will',
 'love',
 'after',
 'all',
 'is',
 'all',
 'is',
 'said',
 'and',
 'done',
 'i',
 'once',
 'believed',
 'that',
 'hearts',
 'were',
 'made',
 'to',
 'bleed',
 'inside',
 'i',
 'once',
 'believed',
 'that',
 'hearts',
 'were',
 'made',
 'to',
 'bleed',
 'oh',
 'baby',
 'but',
 'now',
 'i',
 'm',
 'not',
 'afraid',
 'to',
 'say',
 'i',
 'need',
 'you',
 'i',
 'need',
 'you',
 'so',
 'stay',
 'with',
 'me',
 'these',
 'precious',
 'precious',
 'hours',
 'yeah',
 'greet',
 'each',
 'dawn',
 'in',
 'open',
 

### Stopwords

Stopwords são palavras que, apesar de muito frequentes, não são importantes/relevantes para a máquina. Entre elas, podemos encontrar artigos como “o” e “uma”, ou preposições como “de” e “em”, entre outras palavras frequentes no idioma. Para removê-las do texto, utilizamos uma lista de stopwords disponível na biblioteca NLTK.

In [7]:
from nltk.corpus import stopwords
stops = stopwords.words('english')
stops #lista de stopwords em inglês 

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

Como remover stopwords:

In [8]:
sem_stopwords = [palavra for palavra in letras if palavra not in stops]
palavras_importantes = " ".join(sem_stopwords)
palavras_importantes

'looking mirror open face pain erased sky clearer see sun said done oh always strong need let give live fearless protected one love said done believed hearts made bleed inside believed hearts made bleed oh baby afraid say need need stay precious precious hours yeah greet dawn open arms dream tomorrow love said done yeah baby oh baby inside believed hearts meant bleed never never afraid say need need still moment fear gone hope lives found happy ending love love sweet sweet love said done yeah baby said done'

### Lematização 

Assim como Stopwords, ter verbos conjugados em um texto não faz diferença quando a máquina vai processá-lo. Por isso, existem duas ferramentas chamadas Lemmatização e Stemmatização. Ambas fazem a mesma coisa: Quando passado um texto como argumento, elas reduzem todas as formas verbais conjugadas à sua raiz. A única diferença, entretanto, é que a função que lemmatiza seu texto reduz todos os verbos a forma verdadeira da raiz  -  por isso quanto maior seu texto, mais tempo essa função demora para rodar no código - , enquanto a função que stemmatiza apenas "corta" as palavras no meio usando a raiz como base, o que pode gerar palavras que não existem.

In [9]:
import spacy
spc = spacy.load('en_core_web_sm')

In [10]:
spc_letras = spc(palavras_importantes)
lemmas = [token.lemma_ if token.pos_ == 'VERB' else str(token) for token in spc_letras]
texto_limpo = " ".join(lemmas)
print(texto_limpo)

look mirror open face pain erase sky clearer see sun say do oh always strong need let give live fearless protect one love say do believe hearts make bleed inside believe hearts make bleed oh baby afraid say need need stay precious precious hours yeah greet dawn open arms dream tomorrow love say do yeah baby oh baby inside believe hearts mean bleed never never afraid say need nee still moment fear go hope lives find happy end love love sweet sweet love say do yeah baby say do


Vamos construir uma função para realizar todos os pré-processamentos ao invés de fazê-los um a um: 

In [11]:
def limpar_texto(texto):

    # selecionando apenas as letras e convertendo para minúscula 
    letras =  re.findall(r'\b[A-zÀ-úü]+\b', texto.lower())
    
    # removendo as stopwords 
    stops = set(stopwords.words('portuguese'))
    palavras = [w for w in letras if w not in stops]
    palavras_importantes = " ".join(palavras)
    
    # lematização 
    spc_letras = spc(palavras_importantes)
    lemmas = [token.lemma_ if token.pos_ == 'VERB' else str(token) for token in spc_letras]
    texto_limpo = " ".join(lemmas)
    
    return texto_limpo 

Agora vamos aplicá-la aos nossos dados, mais espcificamente na coluna "letra", que contém as músicas:

In [12]:
df['Texto Limpo'] = df['letra'].apply(limpar_texto)

In [13]:
df.head() # vamos ver como ficou?

,Nome da Música,link,album,letra,artista,Texto Limpo
0,'03 Bonnie & Clyde,/beyonce/03-bonnie-clyde.html,I Am... Yours: An Intimate Performance at Wynn...,Jay-z Uh-uh-uh You ready b? Let's go get 'em. ...,Beyoncé,jay z uh uh uh you ready b let s go get look y...
1,***Flawless (Feat. Chimamanda Ngozi Adichie),/beyonce/flawless-feat-chimamanda-ngozi-adichi...,BEYONCÉ,Your challengers are a young group from Housto...,Beyoncé,your challengers are young group from houston ...
2,***Flawless (Feat. Nicki Minaj),/beyonce/flawless-feat-nicki-minaj.html,BEYONCÉ [Platinum Edition],"Dum-da-de-da Do, do, do, do, do, do (Coming do...",Beyoncé,dum come down drip candy on the ground it stay...
3,1+1,/beyonce/11.html,BEYONCÉ [Platinum Edition],If I ain't got nothing I got you If I ain't go...,Beyoncé,if i ain t get nothing i get you if i ain t ge...
4,6 Inch (Feat. The Weeknd),/beyonce/6-inch-feat-the-weeknd.html,LEMONADE,Six inch heels She walked in the club like nob...,Beyoncé,six inch heels she walk in the club like nobod...


## Feature Extraction
Antes de treinar o nosso modelo, precisamos organizar os nossos documentos em features que o computador consegue entender, assim, vamos precisamos transformar o nosso texto em algum tipo de representação numérica. Para isso, vamos usar o Bag of Words. 

### Bag of Words 
**O que é o Bag of Words?:** BoW é uma forma de representação de texto que descreve a ocorrência de palavras em um documento. Para o BoW a ordem não importa, essa forma de representação só se importa se as palavras conhecidas ocorrem ou não no documento (literalmente um "saco" de palavras). 

Para implementarmos o Bag of Words, precisamos de três coisas: 
1. Um vocabulário com as palavras conhecidas
2. A ocorrência dessas palavras
3. Formar vetores a partir dos documentos 

**Exemplo**

"to the left to the left everything you own in the box to the left"

1. Construir o vocabulário

    ["to", "the", "left", "everything", "you", "own", "in", "box"]
    

2. Ocorrência das palavras

    {"to": 3, "the": 3, "left":3, "everything":1, "you":1, "own":1, "in":1, "box":1}


3. Vetores

    Considerando que o nosso documento fosse: "to the left to the left"

    Usando o vocabulário que construímos antes, o nosso vetor seria: 

    [2, 2, 2, 0, 0, 0, 0]

### Count Vectorizer 
Felizmente, temos o CountVectorizer! Com ele, conseguimos implementar todos os passos acima de uma maneira bem simples: 

In [14]:
from sklearn.feature_extraction.text import CountVectorizer 

# Bag of words
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(df['Texto Limpo'])

Olhando o nosso vocabulário: 

In [15]:
count_vectorizer.get_feature_names() #Todas as palavras do nosso vocabulário 

['aa',
 'aaaaaah',
 'aaah',
 'aah',
 'aahhhh',
 'aaron',
 'abandon',
 'abanenkani',
 'abaziyo',
 'abit',
 'abita',
 'able',
 'aboard',
 'about',
 'above',
 'abrasive',
 'absolutely',
 'abstain',
 'abu',
 'abunch',
 'abuse',
 'acabado',
 'acabo',
 'acabó',
 'acaso',
 'accent',
 'accept',
 'accepte',
 'acceptin',
 'access',
 'accidentally',
 'accomodation',
 'accomplishments',
 'account',
 'accountant',
 'accule',
 'accusations',
 'ace',
 'ache',
 'achetant',
 'achieve',
 'achètera',
 'acompañarme',
 'across',
 'act',
 'actin',
 'acting',
 'action',
 'activité',
 'actor',
 'actress',
 'actual',
 'actually',
 'acuerdo',
 'ad',
 'add',
 'addict',
 'addicted',
 'addiction',
 'addictive',
 'address',
 'adichie',
 'adicto',
 'adiós',
 'adjust',
 'adlibs',
 'admire',
 'admit',
 'admittin',
 'adolescent',
 'adonde',
 'adoration',
 'adore',
 'adorent',
 'adrenaline',
 'adult',
 'advance',
 'advanced',
 'advantage',
 'advice',
 'advise',
 'affair',
 'affect',
 'affectin',
 'affection',
 'affectio

In [16]:
count_vectorizer.vocabulary_.get('love')

3543

Exemplo da nossa matriz termo-documento:

In [17]:
df_cv = pd.DataFrame(X.toarray(), columns = count_vectorizer.get_feature_names())
df_cv.head()

,aa,aaaaaah,aaah,aah,aahhhh,aaron,abandon,abanenkani,abaziyo,abit,...,égaux,élever,élu,éléverons,état,été,évite,évoque,êt,única
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


No dataframe acima, cada uma das colunas representa uma das palavras do nosso vocabulário, e cada linha, um dos nossos documentos, ou seja, uma das nossas músicas. 

Como são muitas colunas, vamos inspecionar apenas dez delas:  

In [18]:
df_cv.iloc[:, 10:20]

,abita,able,aboard,about,above,abrasive,absolutely,abstain,abu,abunch
0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0
3,0,0,0,3,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
503,0,0,0,1,0,0,0,0,0,0
504,0,0,0,14,0,0,0,0,0,0
505,0,0,0,0,0,0,0,0,0,0
506,0,0,0,0,0,0,0,0,0,0


Podemos observar que a palavra "about" aparece 14 vezes no documento 504. Vamos investigar isso: 

In [19]:
df.iloc[504, 5]

'you the one that i dream about all day you the one that i think about always you are the one so i make sure i behave my love is your love your love is my love baby i love you i need you here with all the time ime baby we mean to be you get smile all the time ime cause you know how to give that you know how to pull back when i go runnin runnin tryin to get away from love ya you know how to love hard i win t lie i m fall hard yep i m fall ya but there s nothin wrong with that you the one that i dream about all day you the one that i think about always you are the one so i make sure i behave my love is your love your love is my love you the one that i dream about all day you the one that i think about always you are the one so i make sure i behave my love is your love your love is mine baby come take now hold now make come alive i have you get the sweetest touch i m so happy you come in my life life cause you know how to give that you know how to pull back when i go runnin runnin tryin t

## Separando em Treino e Teste

In [20]:
from sklearn.model_selection import train_test_split

X = X.toarray()
y = df['artista']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Naive Bayes


In [21]:
from sklearn.naive_bayes import MultinomialNB

#Criando o Modelo Naive Bayes 
naive_bayes = MultinomialNB()

#.......Treinando o Modelo.......
naive_bayes.fit(X_train, y_train)

#Fazendo as previsões
naive_bayes_pred = naive_bayes.predict(X_test)

## Métricas 
Após estarmos com nosso modelo de classificação pronto, devemos avaliá-lo e, para isso, utilizamos as métricas de classificação. 
### Matriz de confusão
Primeiro, quando estamos lidando com um modelo cuja target é categórica (como no nosso caso, em que as músicas pertencem ou a Beyoncé ou a Rihanna), podemos utilizar uma matriz de confusão para analisarmos melhor onde o nosso modelo está acertando e onde ele está errando. Ela apresenta o seguinte formato:

<img src="https://www.researchgate.net/profile/Fabio_Araujo_Da_Silva/publication/323369673/figure/fig5/AS:597319787479040@1519423543307/Figura-13-Exemplo-de-uma-matriz-de-confusao.png" alt="Exemplo de uma matriz de confusão"/></a>

Na vertical, estão indicados os valores previstos pelo modelo e, na horizontal, os valores reais. Para cada elemento da matriz, temos dois valores associados: o previsto e o real. Se esse valores coincidirem, tem-se uma previsão correta/verdadeira (por exemplo, verdadeiros positivos e verdadeiros negativos, que estão em verde na imagem). Caso contrário, tem-se um erro cometido pelo modelo (como ocorre nos quadrados vermelhos da imagem acima). 

### Acurácia
A acurácia é, basicamente, uma métrica que indica a relação entre quanto o seu modelo acertou do quanto ele avaliou. Considerando a matriz de confusão mostrada, a acurácia seria igual à soma dos verdadeiros positivos com os verdadeiros negativos dividida pelo total (soma dos verdadeiros e falsos positivos e negativos). A acurácia não é uma boa métrica a ser utilizada quando analisamos dados desbalanceados, porque pode acontecer de o modelo prever muito bem o evento mais usual e ser péssimo prevendo o evento raro. Assim, como trata-se de uma média simples de acertos pelo total, a grande quantidade de acertos na previsão do evento mais usual compensaria a baixa taxa de acerto do evento raro, resultando em uma acurácia alta que não reflete corretamente a qualidade de predição do modelo.

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix

#Calculando a acurácia
acc = accuracy_score(naive_bayes_pred, y_test)

#Matriz de confusão 
cm = confusion_matrix(naive_bayes_pred, y_test)

print("Acurácia do modelo", acc)
print("\nMatriz de confusão: \n", cm)

Acurácia do modelo 0.696078431372549

Matriz de confusão: 
 [[39 12]
 [19 32]]


## Avaliando as músicas
- Tiramos as músicas "Drunk in Love" (Beyoncé) do dataset para testarmos na mão se o modelo consegue prever corretamente: 

In [23]:
frase_beyonce = ["i ve been drinking i ve been drinking i get filthy when that liquor gets into i ve been thinking i ve been thinking why can t i keep my fingers off you baby i want you why can t i keep my fingers off you baby i want you cigars on ice cigars on ice feeling like an animal with these cameras all in my grill flashing lights flashing lights you got faded faded faded baby i want you can t keep your eyes off my fatty daddy i want you drunk in love drunk in love we be all night last thing i remember is our beautiful bodies grinding off in that club drunk in love we be all night love love we be all night love we be all night and everything alright complaints cause my body so fluorescent under these lights boy i m drinking walking in my l assemblage i m grubbing on the rope grubbing if you scared call that reverend boy i m drinking get my brain right i m on the cognac gangster wife new sheets he d swear that i like washed rags he wet up boy i m drinking i m sinking on the mic til my boy toys then i fill the tub up halfway then ride it with my surfboard surfboard surfboard graining on that wood graining graining on that wood i m swerving on that swerving swerving on that big body benz serving all this swerv surfing all of this good good drunk in love we be all night last thing i remember is our beautiful bodies grinding off in that club drunk in love we be all night love love we be all night love love"] 
teste_b = count_vectorizer.transform(frase_beyonce)
pred_b = naive_bayes.predict(teste_b)
print(pred_b)

['Beyoncé']
